In [ ]:
%load_ext autoreload
%autoreload 2
%env CUDA_LAUNCH_BLOCKING=1
import torch

from main import init_model, init_training, create_connections_datasets

In [ ]:
tokenizer, model = init_model()

In [3]:
train_dataset, _ = create_connections_datasets(tokenizer)

In [5]:
training_objects = init_training(model, train_dataset)

In [10]:
from main import init_training
from grpo import rollout, update_policy

from tasks.countdown import reward_function

to = init_training(model, train_dataset)
batch = next(iter(to.train_dataloader))

episodes = rollout(
    model=model,
    tokenizer=tokenizer,
    batch=batch,
    max_gen_len=to.config.max_gen_len,
    num_answer_per_question=to.config.num_answer_per_question,
    reward_function=reward_function,
    device=to.device,
)
if to.config.skip_unfinished_episodes:
    episodes = [episode for episode in episodes if episode.is_finished]

2025-05-04 13:11:18.569 | INFO     | grpo:rollout:43 - Generating responses for 1 prompts, max_gen_len=100
2025-05-04 13:11:39.310 | INFO     | grpo:rollout:73 - Generated token ids: 100
2025-05-04 13:11:39.311 | INFO     | grpo:rollout:80 - Generated token ids after removing padding: 100
2025-05-04 13:11:39.312 | INFO     | grpo:rollout:87 - Generated text: 

Okay, let's see. The user provided a list of words: pinch, rob, steal, swipe, sen, sight, sine, sour, dogleg, hairpin, switchback, zag, infant, pub, swimming, ticker. They want me to group them into four groups of four words each, using each word only once per group. 

First, I need to look for obvious connections. Let's start with "sine" and "sight" because
2025-05-04 13:11:39.314 | INFO     | __main__:<module>:21 - Updating policy for step 1


In [15]:
episodes[0].is_finished

False

In [27]:
# Update policy - compute loss and perform backward pass
results = update_policy(
    model=model,
    optimizer=to.optimizer,
    episodes=episodes,
    micro_batch_size=to.config.micro_batch_size,
    pad_token_id=tokenizer.pad_token_id,
    max_grad_norm=to.config.max_grad_norm,
    device=to.device,
    dtype=to.dtype,
)
if torch.cuda.is_available():
    torch.cuda.synchronize()



2025-05-04 13:14:56.235 | INFO     | grpo:update_policy:7 - Updating policy with 1 episodes, 100 target tokens, 1 micro-batches
* Computing policy gradient:  0/ 1mINFO     | grpo:update_policy:9 - 


[[151644, 8948, 271, 2610, 525, 458, 6203, 24626, 21828, 1614, 624, 9885, 5203, 315, 4244, 429, 525, 5435, 311, 1817, 1008, 13, 8886, 1874, 374, 3040, 4244, 1293, 13, 2619, 525, 6896, 3040, 5203, 304, 2790, 624, 2610, 1231, 1172, 990, 1817, 3409, 304, 825, 1874, 624, 65354, 304, 279, 2701, 3561, 510, 27, 19895, 287, 397, 9338, 522, 19895, 287, 397, 27, 9217, 397, 27, 4074, 397, 9338, 522, 4074, 397, 27, 4074, 397, 9338, 522, 4074, 397, 522, 9217, 397, 2, 13383, 271, 1474, 25, 37799, 11, 88916, 263, 11, 25744, 21873, 11, 25349, 11, 9055, 11, 27291, 11, 700, 11, 2874, 11, 595, 7066, 642, 11, 47332, 724, 11, 11174, 90512, 11, 40659, 388, 11, 74982, 11, 282, 524, 11, 296, 7417, 11, 259, 32779, 198, 71703, 25, 366, 19895, 287, 397, 40, 3278, 1191, 448, 14719, 1495, 279, 32711, 13, 1752, 5737, 220, 16, 11, 279, 4244, 5435, 311, 36023, 25, 37799, 11, 88916, 263, 11, 25744, 21873, 11, 25349, 1959, 20913, 311, 36023, 304, 5257, 7586, 11, 1741, 438, 88916, 263, 323, 51205, 1660, 36023, 5980, 624

In [17]:
results

{'loss': nan, 'grad_norm': nan, 'entropy': 0.31090065836906433}